In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import numpy as np

from matplotlib import pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [2]:
def generate_connected_graph(cluster_sizes=[100, 100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probabilities = np.array([[pin, pout], [pout, pin]])
    while True:
        graph = nx.stochastic_block_model(cluster_sizes, probabilities)
        if nx.algorithms.components.is_connected(graph):
            return graph

def visualize_graph(graph):
    nx.draw(graph, with_labels=True, node_size=100, alpha=1, linewidths=10)
    plt.show()

# Parameters
cluster_sizes = [10, 10]
pin = 0.5
pout = 0.2
seed = 0
alpha = 1e-2
lamda = 1e-3
eta = 1e-3
no_users = 20#sum(cluster_sizes)
batch_size = 64
epochs = 1
embedding_dimension = 3372
iterations = 2000

# Generate a binomial graph with 20 nodes and probability of edge creation p=0.2
G = nx.binomial_graph(n=no_users, p=0.3, seed=0)#generate_connected_graph(cluster_sizes=cluster_sizes, pin=0.5, pout=0.01, seed=0)
#nx.binomial_graph(n=no_users, p=0.1, seed=0)
#visualize_graph(graph)

In [3]:
# Metropolis weights 
number_nodes = G.number_of_nodes()
weights = np.zeros([number_nodes, number_nodes])
for edge in G.edges():
  i, j = edge[0], edge[1]
  weights[i - 1][j - 1] = 1 / (1 + np.max([G.degree(i), G.degree(j)]))
  weights[j - 1][i - 1] = weights[i - 1][j - 1]

print(weights)

weights = weights + np.diag(1 - np.sum(weights, axis=0))

metropolis_weights = weights
print(metropolis_weights)


[[0.         0.         0.         0.         0.         0.
  0.         0.1        0.         0.         0.         0.
  0.         0.         0.25       0.         0.         0.1
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.1
  0.         0.         0.11111111 0.         0.         0.16666667
  0.         0.14285714 0.         0.         0.         0.1
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.1
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.14285714 0.        ]
 [0.         0.         0.         0.         0.2        0.1
  0.         0.         0.         0.14285714 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.2       ]
 [0.         0.         0.         0.2        0.         0.1
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         

In [4]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [5]:
# Dataset partitioning
def random_split(X, y, n, seed):
    """Equally split data between n agents"""
    rng = np.random.default_rng(seed)
    perm = rng.permutation(y.size)
    X_split = np.array_split(X[perm], n)  #np.stack to keep as a np array
    y_split = np.array_split(y[perm], n)
    return X_split, y_split




'''
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

no_features = X_train.shape[1]

X, y = random_split(X_train, y_train, no_users, 1234)
'''


train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

concatenated_df = pd.concat([train_data, test_data], axis=0)

# Display the concatenated DataFrame
# Replace activities not in the 'keep_activities' list with 'OTHER_ACTIVITIES'
#concatenated_df['Activity'] = np.where(concatenated_df['Activity'].isin(keep_activities), concatenated_df['Activity'], 'OTHER_ACTIVITIES')

# Split the data into training and testing sets
train_data, test_data = train_test_split(concatenated_df, test_size=0.2, random_state=42)


keep_activities = ['SITTING']





x_train, y_train = train_data.iloc[:, :-2], train_data.iloc[:, -1:]
x_test, y_test = test_data.iloc[:, :-2], test_data.iloc[:, -1:]
x_train.shape, y_train.shape

x_test, y_test = test_data.iloc[:, :-2], test_data.iloc[:, -1:]
x_test.shape, y_test.shape

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

scaling_data = MinMaxScaler()
x_train = scaling_data.fit_transform(x_train)
X_test = scaling_data.transform(x_test)

train_list = []

for group_value in train_data['subject'].unique():
    print(group_value)
    # Query the DataFrame for the specific group
    group_data = train_data.query(f'subject == {group_value}')
    x_train, y_train = group_data.iloc[:, :-2], group_data.iloc[:, -1:]
    y_train = le.fit_transform(y_train)
    x_train = scaling_data.fit_transform(x_train)

    
    # Extract the 'value' column and convert it to a NumPy array
    train_list.append((x_train, y_train))




no_features = X_test.shape[1]

#X, y = random_split(X_train, y_train, no_users, 1234)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


5
19
13


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

4
6
22
7
18
26
14
24


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


28
8


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


12
11


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
datapoints = {}
count = 0


scaler = [1.0, -1.0]

noise_sd = 0.001
for i in range(no_users):
    #features = np.random.normal(loc=0.0, scale=1.0, size=(m, n))
    #label = np.dot(features, W[i ]) + np.random.normal(0,noise_sd)
    data = train_list[i][0]
    #data[:, 0:no_features//2] *= scaler[i]
    datapoints[count] = {
            'features': data,
            'degree': node_degree(i, G),
            'label': train_list[i][1],
            'neighbors': get_neighbors(i, G),
            #'exact_weights': torch.from_numpy(W[i])
        }
    count += 1

In [7]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.LongTensor(targets)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [8]:
'''
class MLP_Net(nn.Module):
    def __init__(self, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(no_features, 64, bias=True)
        self.fc2 = nn.Linear(64, 6, bias=True)
        #self.fc3 = nn.Linear(6, 10)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        output = F.softmax(x, dim=0)
        return output
'''

'\nclass MLP_Net(nn.Module):\n    def __init__(self, user_id):\n        super(MLP_Net, self).__init__()\n        self.fc1 = nn.Linear(no_features, 64, bias=True)\n        self.fc2 = nn.Linear(64, 6, bias=True)\n        #self.fc3 = nn.Linear(6, 10)\n        self.user_id = user_id\n\n    def forward(self, x):\n        x = torch.flatten(x, 1)\n        x = F.relu(self.fc1(x))\n        x = self.fc2(x)\n        output = F.softmax(x, dim=0)\n        return output\n'

In [9]:
input_size = X_test.shape[1]
hidden_size1 = 64
hidden_size2 = 128
output_size = 6

class MLP_Net(nn.Module):
    def __init__(self, user_id, input_size, hidden_size1, hidden_size2, output_size):
        super(MLP_Net, self).__init__()

        #self.layer1 = nn.Linear(input_size, hidden_size1, bias=True)
        #self.activation1 = nn.ReLU()

        #self.layer2 = nn.Linear(hidden_size1, hidden_size2, bias=True)
        #self.activation2 = nn.ReLU()

        #self.layer3 = nn.Linear(hidden_size2, 64, bias=True)
        #self.activation3 = nn.ReLU()

        self.output_layer = nn.Linear(input_size, output_size, bias=True)
        self.softmax = nn.Softmax(dim=1)
        self.user_id = user_id

    def forward(self, x):
        #x = self.activation1(self.layer1(x))
        #x = self.activation2(self.layer2(x))
        #x = self.activation3(self.layer3(x))
        x = self.softmax(self.output_layer(x))
        return x


In [10]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [11]:
batch_size

64

In [12]:
# User-specific information
user_id = 10

# Initialize the MLP model
model = MLP_Net(user_id, input_size, hidden_size1, hidden_size2, output_size)

# Learning rate
lr = 0.01

# Create a DataLoader for the dataset
dataloader = DataLoader(MyDataset(datapoints[19]["features"], datapoints[19]["label"]), batch_size=64, shuffle=True)

# Define the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=lr)


losses = []

acuracies = []

# Training loop
for epoch in range(1000):
    batch_acc = 0
    batch_s = 0
    sum_loss = 0
    data_points_length = 0
    
    # Iterate over batches in the DataLoader
    for (x, y) in dataloader:
        # Loss function
        criterion = nn.CrossEntropyLoss()

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        yhat = model(x)

        # Calculate accuracy
        pred = torch.max(yhat, dim=1)[1]
        batch_acc += (pred == y).sum().item()
        batch_s += pred.size()[0]

        # Calculate loss
        loss = criterion(yhat, y)

        # Backward pass
        loss.backward()

        # Update the optimizer
        optimizer.step()

        # Accumulate loss and data point count
        sum_loss += loss.detach().item()
        data_points_length += yhat.size()[0]
    losses.append(sum_loss / data_points_length)
    acuracies.append(batch_acc / batch_s)

    # Print training statistics
    print(f"Epoch {epoch + 1}/{1000}: Accuracy = {batch_acc / batch_s:.7f}, Loss = {sum_loss / data_points_length:.7f}")


Epoch 1/1000: Accuracy = 0.1699346, Loss = 0.0293911
Epoch 2/1000: Accuracy = 0.2418301, Loss = 0.0293027
Epoch 3/1000: Accuracy = 0.2908497, Loss = 0.0292206
Epoch 4/1000: Accuracy = 0.3039216, Loss = 0.0291294
Epoch 5/1000: Accuracy = 0.3104575, Loss = 0.0290523
Epoch 6/1000: Accuracy = 0.3137255, Loss = 0.0289576
Epoch 7/1000: Accuracy = 0.3137255, Loss = 0.0288702
Epoch 8/1000: Accuracy = 0.3137255, Loss = 0.0287932
Epoch 9/1000: Accuracy = 0.3137255, Loss = 0.0287121
Epoch 10/1000: Accuracy = 0.3137255, Loss = 0.0286189
Epoch 11/1000: Accuracy = 0.3202614, Loss = 0.0285435
Epoch 12/1000: Accuracy = 0.3267974, Loss = 0.0284632
Epoch 13/1000: Accuracy = 0.3333333, Loss = 0.0283755
Epoch 14/1000: Accuracy = 0.3529412, Loss = 0.0282943
Epoch 15/1000: Accuracy = 0.3692810, Loss = 0.0282062
Epoch 16/1000: Accuracy = 0.4117647, Loss = 0.0281285
Epoch 17/1000: Accuracy = 0.4444444, Loss = 0.0280582
Epoch 18/1000: Accuracy = 0.4738562, Loss = 0.0279722
Epoch 19/1000: Accuracy = 0.4771242, 

Epoch 155/1000: Accuracy = 0.8071895, Loss = 0.0217357
Epoch 156/1000: Accuracy = 0.8071895, Loss = 0.0217717
Epoch 157/1000: Accuracy = 0.8071895, Loss = 0.0216855
Epoch 158/1000: Accuracy = 0.8071895, Loss = 0.0216876
Epoch 159/1000: Accuracy = 0.8071895, Loss = 0.0216609
Epoch 160/1000: Accuracy = 0.8071895, Loss = 0.0216294
Epoch 161/1000: Accuracy = 0.8071895, Loss = 0.0216690
Epoch 162/1000: Accuracy = 0.8071895, Loss = 0.0215714
Epoch 163/1000: Accuracy = 0.8071895, Loss = 0.0215957
Epoch 164/1000: Accuracy = 0.8071895, Loss = 0.0215462
Epoch 165/1000: Accuracy = 0.8071895, Loss = 0.0215840
Epoch 166/1000: Accuracy = 0.8071895, Loss = 0.0215726
Epoch 167/1000: Accuracy = 0.8071895, Loss = 0.0215374
Epoch 168/1000: Accuracy = 0.8071895, Loss = 0.0215124
Epoch 169/1000: Accuracy = 0.8071895, Loss = 0.0215190
Epoch 170/1000: Accuracy = 0.8071895, Loss = 0.0214778
Epoch 171/1000: Accuracy = 0.8071895, Loss = 0.0214956
Epoch 172/1000: Accuracy = 0.8071895, Loss = 0.0214498
Epoch 173/

Epoch 312/1000: Accuracy = 0.9869281, Loss = 0.0198118
Epoch 313/1000: Accuracy = 0.9869281, Loss = 0.0197874
Epoch 314/1000: Accuracy = 0.9869281, Loss = 0.0197601
Epoch 315/1000: Accuracy = 0.9869281, Loss = 0.0197798
Epoch 316/1000: Accuracy = 0.9869281, Loss = 0.0197577
Epoch 317/1000: Accuracy = 0.9869281, Loss = 0.0197621
Epoch 318/1000: Accuracy = 0.9901961, Loss = 0.0197600
Epoch 319/1000: Accuracy = 0.9869281, Loss = 0.0197278
Epoch 320/1000: Accuracy = 0.9901961, Loss = 0.0197191
Epoch 321/1000: Accuracy = 0.9869281, Loss = 0.0197177
Epoch 322/1000: Accuracy = 0.9869281, Loss = 0.0197086
Epoch 323/1000: Accuracy = 0.9869281, Loss = 0.0197110
Epoch 324/1000: Accuracy = 0.9869281, Loss = 0.0197216
Epoch 325/1000: Accuracy = 0.9869281, Loss = 0.0196994
Epoch 326/1000: Accuracy = 0.9901961, Loss = 0.0196748
Epoch 327/1000: Accuracy = 0.9869281, Loss = 0.0196873
Epoch 328/1000: Accuracy = 0.9901961, Loss = 0.0196841
Epoch 329/1000: Accuracy = 0.9901961, Loss = 0.0196708
Epoch 330/

Epoch 471/1000: Accuracy = 0.9967320, Loss = 0.0189275
Epoch 472/1000: Accuracy = 0.9967320, Loss = 0.0189202
Epoch 473/1000: Accuracy = 0.9967320, Loss = 0.0189127
Epoch 474/1000: Accuracy = 0.9967320, Loss = 0.0189035
Epoch 475/1000: Accuracy = 0.9967320, Loss = 0.0189091
Epoch 476/1000: Accuracy = 0.9967320, Loss = 0.0189077
Epoch 477/1000: Accuracy = 0.9967320, Loss = 0.0189075
Epoch 478/1000: Accuracy = 0.9967320, Loss = 0.0188884
Epoch 479/1000: Accuracy = 0.9967320, Loss = 0.0188945
Epoch 480/1000: Accuracy = 0.9967320, Loss = 0.0188826
Epoch 481/1000: Accuracy = 0.9967320, Loss = 0.0188787
Epoch 482/1000: Accuracy = 0.9967320, Loss = 0.0188839
Epoch 483/1000: Accuracy = 0.9967320, Loss = 0.0188787
Epoch 484/1000: Accuracy = 0.9967320, Loss = 0.0188668
Epoch 485/1000: Accuracy = 0.9967320, Loss = 0.0188706
Epoch 486/1000: Accuracy = 0.9967320, Loss = 0.0188728
Epoch 487/1000: Accuracy = 0.9967320, Loss = 0.0188602
Epoch 488/1000: Accuracy = 0.9967320, Loss = 0.0188598
Epoch 489/

Epoch 626/1000: Accuracy = 0.9967320, Loss = 0.0184906
Epoch 627/1000: Accuracy = 0.9967320, Loss = 0.0184715
Epoch 628/1000: Accuracy = 0.9967320, Loss = 0.0184828
Epoch 629/1000: Accuracy = 0.9967320, Loss = 0.0184585
Epoch 630/1000: Accuracy = 0.9967320, Loss = 0.0184591
Epoch 631/1000: Accuracy = 0.9967320, Loss = 0.0184561
Epoch 632/1000: Accuracy = 0.9967320, Loss = 0.0184555
Epoch 633/1000: Accuracy = 0.9967320, Loss = 0.0184666
Epoch 634/1000: Accuracy = 0.9967320, Loss = 0.0184522
Epoch 635/1000: Accuracy = 0.9967320, Loss = 0.0184594
Epoch 636/1000: Accuracy = 0.9967320, Loss = 0.0184722
Epoch 637/1000: Accuracy = 0.9967320, Loss = 0.0184666
Epoch 638/1000: Accuracy = 0.9967320, Loss = 0.0184468
Epoch 639/1000: Accuracy = 0.9967320, Loss = 0.0184500
Epoch 640/1000: Accuracy = 0.9967320, Loss = 0.0184467
Epoch 641/1000: Accuracy = 0.9967320, Loss = 0.0184379
Epoch 642/1000: Accuracy = 0.9967320, Loss = 0.0184466
Epoch 643/1000: Accuracy = 0.9967320, Loss = 0.0184509
Epoch 644/

Epoch 782/1000: Accuracy = 1.0000000, Loss = 0.0182101
Epoch 783/1000: Accuracy = 1.0000000, Loss = 0.0182052
Epoch 784/1000: Accuracy = 1.0000000, Loss = 0.0182029
Epoch 785/1000: Accuracy = 1.0000000, Loss = 0.0181971
Epoch 786/1000: Accuracy = 1.0000000, Loss = 0.0182088
Epoch 787/1000: Accuracy = 1.0000000, Loss = 0.0182077
Epoch 788/1000: Accuracy = 1.0000000, Loss = 0.0181967
Epoch 789/1000: Accuracy = 1.0000000, Loss = 0.0181944
Epoch 790/1000: Accuracy = 1.0000000, Loss = 0.0181989
Epoch 791/1000: Accuracy = 1.0000000, Loss = 0.0181939
Epoch 792/1000: Accuracy = 1.0000000, Loss = 0.0181957
Epoch 793/1000: Accuracy = 1.0000000, Loss = 0.0181873
Epoch 794/1000: Accuracy = 1.0000000, Loss = 0.0181829
Epoch 795/1000: Accuracy = 1.0000000, Loss = 0.0181813
Epoch 796/1000: Accuracy = 1.0000000, Loss = 0.0181859
Epoch 797/1000: Accuracy = 1.0000000, Loss = 0.0181825
Epoch 798/1000: Accuracy = 1.0000000, Loss = 0.0181780
Epoch 799/1000: Accuracy = 1.0000000, Loss = 0.0181785
Epoch 800/

Epoch 933/1000: Accuracy = 1.0000000, Loss = 0.0180104
Epoch 934/1000: Accuracy = 1.0000000, Loss = 0.0180256
Epoch 935/1000: Accuracy = 1.0000000, Loss = 0.0180315
Epoch 936/1000: Accuracy = 1.0000000, Loss = 0.0180344
Epoch 937/1000: Accuracy = 1.0000000, Loss = 0.0180144
Epoch 938/1000: Accuracy = 1.0000000, Loss = 0.0180315
Epoch 939/1000: Accuracy = 1.0000000, Loss = 0.0180232
Epoch 940/1000: Accuracy = 1.0000000, Loss = 0.0180177
Epoch 941/1000: Accuracy = 1.0000000, Loss = 0.0180255
Epoch 942/1000: Accuracy = 1.0000000, Loss = 0.0180187
Epoch 943/1000: Accuracy = 1.0000000, Loss = 0.0180096
Epoch 944/1000: Accuracy = 1.0000000, Loss = 0.0180238
Epoch 945/1000: Accuracy = 1.0000000, Loss = 0.0180349
Epoch 946/1000: Accuracy = 1.0000000, Loss = 0.0180237
Epoch 947/1000: Accuracy = 1.0000000, Loss = 0.0180097
Epoch 948/1000: Accuracy = 1.0000000, Loss = 0.0180169
Epoch 949/1000: Accuracy = 1.0000000, Loss = 0.0180270
Epoch 950/1000: Accuracy = 1.0000000, Loss = 0.0180196
Epoch 951/

In [13]:
'''
# Plotting Accuracies
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
plt.plot(acuracies, label='Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plotting Losses
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
plt.plot(losses, label='Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()  # Ensures proper spacing between subplots
plt.show()
'''

"\n# Plotting Accuracies\nplt.figure(figsize=(12, 6))\n\nplt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot\nplt.plot(acuracies, label='Accuracy')\nplt.title('Accuracy Over Epochs')\nplt.xlabel('Epoch')\nplt.ylabel('Accuracy')\nplt.legend()\n\n# Plotting Losses\nplt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot\nplt.plot(losses, label='Loss')\nplt.title('Loss Over Epochs')\nplt.xlabel('Epoch')\nplt.ylabel('Loss')\nplt.legend()\n\nplt.tight_layout()  # Ensures proper spacing between subplots\nplt.show()\n"

In [14]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.5)

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                grads = grads_to_vector(model.parameters())
                #optimizer.step()
                train_loss += loss.item()*data.size(0)
                weights = parameters_to_vector(model.parameters())
                mat_vec_sum = torch.zeros_like(weights)
                for j in G.neighbors(model.user_id):
                    mat_vec_sum = torch.add(mat_vec_sum, torch.matmul(torch.transpose(projection_list[model.user_id][j], 0, 1), 
                                                         projected_weights[model.user_id][j] - projected_weights[j][model.user_id]))
                
                model_update = parameters_to_vector(model.parameters()) - alpha * (grads + lamda * mat_vec_sum)
                
            vector_to_parameters(parameters=model.parameters(), vec=model_update)
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [15]:
# Preparing projection matrices
models = [MLP_Net(i, input_size, hidden_size1, hidden_size2, output_size) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, first_run=True):
    #projected_weights = []
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        row, column = embedding_dimension, parameters_to_vector(models[i].parameters()).size()[0]
                        mat = torch.zeros((row, column))
                        mat.fill_diagonal_(1.0 + 1.0 * float(np.random.randn(1)))
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[i].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[i][j], parameters_to_vector(models[i].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights)



In [16]:
total_params = sum(p.numel() for p in models[0].parameters())
total_weights = sum(p.numel() for p in models[0].parameters() if p.requires_grad)
total_biases = total_params - total_weights

print(f'Total parameters in the model: {total_params}')
print(f'Total weights in the model: {total_weights}')
print(f'Total biases in the model: {total_biases}')

Total parameters in the model: 3372
Total weights in the model: 3372
Total biases in the model: 0


In [17]:
print(projection_list[0][1].shape)

AttributeError: 'int' object has no attribute 'shape'

In [18]:
def testing(model, dataset, bs, criterion):
    test_loss = 0
    correct = 0
    total_samples = 0

    test_loader = DataLoader(MyDataset(X_test, y_test), batch_size=bs, shuffle=False)
    
    model.eval()

    with torch.no_grad():
        for data, labels in test_loader:
            output = model(data)
            loss = criterion(output, labels)
            test_loss += loss.item() * data.size(0)

            _, pred = torch.max(output, 1)
            correct += pred.eq(labels.data.view_as(pred)).sum().item()
            total_samples += labels.size(0)

    test_loss /= len(test_loader.dataset)
    test_accuracy = correct / total_samples

    return test_loss, test_accuracy

In [19]:
projection_list[0][6].shape

AttributeError: 'int' object has no attribute 'shape'

In [20]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(i, input_size, hidden_size1, hidden_size2, output_size) for i in range(no_users) ]
dummy_models = [MLP_Net(i, input_size, hidden_size1, hidden_size2, output_size) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.CrossEntropyLoss()

it = 100
train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    # Update prjection matrix
    
    #print(projection_list[0], projected_weights[0])
    
    for i in range(no_users):
        weights = parameters_to_vector(models[i].parameters())
        for j in G.neighbors(i):
            mat_vec_sum = torch.zeros(embedding_dimension)
            for k in G.neighbors(i):
                mat_vec_sum = torch.add(mat_vec_sum, projected_weights[i][k] - projected_weights[k][i])
            temp_mat = torch.outer(mat_vec_sum, weights).clone()


            projection_list[i][j] = torch.add(projection_list[i][j], -1 * eta * lamda * temp_mat)
                                         
    projected_weights = []                                          
    update_ProjWeight(projection_list, projected_weights, first_run=False)
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    user_rel_error = 0
    for k in range(no_users):
      
        g_loss, accuracy = testing(models[i], datapoints[i], 50, criterion)
        local_test_loss.append(g_loss)
        #user_rel_error += rel_error(models[i])
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    #total_rel_error.append(user_rel_error / no_users)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.8f,   %2.8f"% (test_loss[-1], accuracy))

  1%|          | 1/100 [00:10<17:03, 10.34s/it]

Training_loss 1.79231809,   0.13446602


  2%|▏         | 2/100 [00:20<16:20, 10.01s/it]

Training_loss 1.79166406,   0.13543689


  3%|▎         | 3/100 [00:30<16:43, 10.35s/it]

...

Training_loss 1.73986080,   0.37524272


 99%|█████████▉| 99/100 [16:34<00:10, 10.87s/it]

Training_loss 1.73924762,   0.38640777


100%|██████████| 100/100 [16:44<00:00, 10.05s/it]

Training_loss 1.73872310,   0.39417476


In [ ]:
#Training_loss 5.33078 with no communication

In [ ]:
#plot.plot(test_loss)
parameters_to_vector(models[19].parameters())

In [ ]:
for j in G.neighbors(0):
    print(j)

In [ ]:
parameters_to_vector(models[0].parameters())

In [ ]:
projection_list[0]

In [ ]:
projected_weights[0]

In [ ]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

In [ ]:
print(test_loss)

In [ ]:
np.save( 'training_loss_sheave_fml_lambda' + str(lamda).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), test_loss)
#np.save('relative_error_sheave_fml' + str(lamda).replace('.', '_'), total_rel_error)

In [ ]:
'training_loss_sheave_fml' + str(lamda).replace('.', '_'), test_loss